# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

### Projeto de Chatbot Personalizado

Escolhi utilizar o dataset `character_descriptions.csv`, que contém descrições de personagens fictícios criados por um modelo da OpenAI. Este conjunto de dados é apropriado para este projeto porque cada linha apresenta uma amostra textual rica e coerente, contendo o nome do personagem, uma descrição detalhada, o meio em que aparece e o ambiente em que a história se passa.

Essas informações são altamente textuais e contextualizadas, o que torna o dataset ideal para treinar um chatbot personalizado capaz de responder a perguntas sobre esses personagens e até sugerir ideias de narrativa.

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [4]:
import pandas as pd

# Carregar o dataset de descrições de personagens
df = pd.read_csv("data/character_descriptions.csv")

# Exibir as primeiras linhas para inspeção
df.head()

,Name,Description,Medium,Setting
0,Emily,"A young woman in her early 20s, Emily is an as...",Play,England
1,Jack,"A middle-aged man in his 40s, Jack is a succes...",Play,England
2,Alice,"A woman in her late 30s, Alice is a warm and n...",Play,England
3,Tom,"A man in his 50s, Tom is a retired soldier and...",Play,England
4,Sarah,"A woman in her mid-20s, Sarah is a free-spirit...",Play,England


In [15]:
df["text"] = df.apply(
    lambda row: f"Name: {row['Name']}. Description: {row['Description']}. Medium: {row['Medium']}. Setting: {row['Setting']}.",
    axis=1
)

# Visualizar as primeiras linhas
df["text"].head()


0    Name: Emily. Description: A young woman in her...
1    Name: Jack. Description: A middle-aged man in ...
2    Name: Alice. Description: A woman in her late ...
3    Name: Tom. Description: A man in his 50s, Tom ...
4    Name: Sarah. Description: A woman in her mid-2...
Name: text, dtype: object

## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [ ]:
!pip install openai==0.26.1

In [18]:
import openai
import getpass

# Configuração da API da OpenAI no ambiente do Vocareum
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = getpass.getpass("Digite sua API Key: ").strip()

In [28]:
# Gerar embeddings para os textos
def get_embedding(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response['data'][0]['embedding']

In [29]:
# Calcula os embeddings (demora um pouco)
df["embedding"] = df["text"].apply(get_embedding)

In [30]:
df.to_csv("data/character_descriptions_embedding.csv")

In [31]:
from sklearn.metrics.pairwise import cosine_similarity


# Função para responder perguntas com base nos textos mais similares
def ask_semantic_bot(question, top_k=3):
    # Gerar embedding da pergunta
    question_embedding = get_embedding(question)

    # Calcular similaridade entre a pergunta e os textos
    similarities = cosine_similarity(
        [question_embedding],
        df["embedding"].tolist()
    )[0]

    # Selecionar os top_k textos mais similares
    top_indices = similarities.argsort()[-top_k:][::-1]
    top_contexts = df.iloc[top_indices]["text"].tolist()

    # Criar prompt com os textos mais relevantes
    prompt = f"""
You are an assistant that answers questions based on fictional character descriptions.

Use the following information to answer the question:

{chr(10).join(top_contexts)}

Question: {question}
Answer:"""

    # Consulta ao modelo
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=300,
        temperature=0.7
    )

    return response["choices"][0]["text"].strip()

In [32]:
question = "Which character would be perfect for a horror series in a haunted hospital?"
answer = ask_semantic_bot(question)

In [33]:
print("Resposta do bot:")
print(answer)

Resposta do bot:
Maya would be perfect for a horror series in a haunted hospital. Her job as a nurse and her compassionate, caring nature would make for an interesting dynamic in a horror setting. Plus, her fearlessness and willingness to stand up for what she believes in would make her a strong protagonist in the series.


## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

In [34]:
# Função que usa apenas a pergunta (sem contexto)
def ask_basic_model(question):
    prompt = f"Answer the following question: {question}"
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=300,
        temperature=0.7
    )
    return response["choices"][0]["text"].strip()


### Question 1

In [38]:
question1 = "Which character and why would be perfect for a sci-fi film?"

print("=== Question 1 ===")
print(f"Question: {question1}\n")

=== Question 1 ===
Question: Which character and why would be perfect for a sci-fi film?



In [39]:
print("Basic model answer:")
print(ask_basic_model(question1), "\n")


Basic model answer:
I cannot answer this question as it is subjective and based on personal opinions. Different people may have different ideas about which character would be perfect for a sci-fi film. Additionally, there are many different types of characters that could be suitable for a sci-fi film, depending on the plot and themes of the film. 



In [40]:
print("Custom prompt answer:")
print(ask_semantic_bot(question1), "\n")

Custom prompt answer:
Sonya would be perfect for a sci-fi film. Her free-spirited nature and passion for seeking out new stories would make her a great fit for a character in a futuristic world. Additionally, her tendency to push people away could add an interesting layer to her character as she navigates the unknown and potentially dangerous world of science fiction. 



### Question 2

In [41]:
question2 = "Who could be a good protagonist for a romantic drama in a small European town and why?"

print("=== Question 2 ===")
print(f"Question: {question2}\n")

=== Question 2 ===
Question: Who could be a good protagonist for a romantic drama in a small European town and why?



In [42]:
print("Basic model answer:")
print(ask_basic_model(question2), "\n")

Basic model answer:
A good protagonist for a romantic drama in a small European town could be a young, independent artist named Sofia. She is a free-spirited and creative soul, who has recently moved to the town to pursue her passion for painting. Sofia is also a kind and compassionate person, who quickly becomes a part of the close-knit community in the town.

As she explores the town and its surroundings, Sofia meets a charming and reserved local baker named Luca. Despite their initial differences, they are drawn to each other and a sweet romance begins to bloom. However, as their relationship deepens, they must navigate through cultural differences and family expectations, adding drama and tension to their love story.

Sofia's artistic nature adds a dreamy and poetic element to the story, while her outsider status in the town allows for an interesting perspective on the traditional values and customs of the community. Her determination to follow her dreams and her willingness to ope

In [43]:
print("Custom prompt answer:")
print(ask_semantic_bot(question2), "\n")

Custom prompt answer:
Prince Lorenzo could be a good protagonist for a romantic drama in a small European town because he is a charming and handsome prince who is searching for true love. His well-traveled and cultured background adds depth to his character and his pursuit of a particular young woman adds a sense of intrigue and determination to the story. Additionally, the setting of Italy adds a romantic and picturesque backdrop for the drama to unfold. 

